In [2]:
import requests
import json
from typing import Dict, List, Optional
import time

import os
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
class KuryanaAPI:
    def __init__(self, base_url: str = "https://kuryana.tbdh.app"):
        self.base_url = base_url
        self.session = requests.Session()
    
    def _make_request(self, endpoint: str) -> Optional[Dict]:
        """Make a request to the API with error handling."""
        try:
            response = self.session.get(f"{self.base_url}{endpoint}")
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            return None
    
    def search_dramas(self, query: str) -> Optional[Dict]:
        """Search for dramas by query string."""
        return self._make_request(f"/search/q/{query}")
    
    def get_drama_info(self, slug: str) -> Optional[Dict]:
        """Get detailed information about a drama."""
        return self._make_request(f"/id/{slug}")
    
    def get_cast(self, slug: str) -> Optional[Dict]:
        """Get cast information for a drama."""
        return self._make_request(f"/id/{slug}/cast")
    
    def get_episodes(self, slug: str) -> Optional[Dict]:
        """Get episode list for a drama."""
        return self._make_request(f"/id/{slug}/episodes")
    
    def get_reviews(self, slug: str) -> Optional[Dict]:
        """Get reviews for a drama."""
        return self._make_request(f"/id/{slug}/reviews")
    
    def get_person_info(self, person_id: str) -> Optional[Dict]:
        """Get information about a person."""
        return self._make_request(f"/people/{person_id}")
    
    def get_seasonal_dramas(self, year: int, quarter: int) -> Optional[Dict]:
        """Get seasonal drama list."""
        return self._make_request(f"/seasonal/{year}/{quarter}")
    
    def get_user_dramalist(self, user_id: str) -> Optional[Dict]:
        """Get user's drama list."""
        return self._make_request(f"/dramalist/{user_id}")
    
    def get_list(self, list_id: str) -> Optional[Dict]:
        """Get a public list."""
        return self._make_request(f"/list/{list_id}")

# Initialize the API client
api = KuryanaAPI()

In [4]:
class KuryanaAPI:
    def __init__(self, base_url: str = "https://kuryana.tbdh.app"):
        self.base_url = base_url
        self.session = requests.Session()
    
    def _make_request(self, endpoint: str) -> Optional[Dict]:
        """Make a request to the API with error handling."""
        try:
            response = self.session.get(f"{self.base_url}{endpoint}")
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            return None
        
    def get_drama_info(self, slug: str) -> Optional[Dict]:
        """Get detailed information about a drama. Primarily to access: slug_query, title, synopsis, genres, tags. Do not use for cast."""
        return self._make_request(f"/id/{slug}")

    def get_cast(self, slug: str) -> Optional[Dict]:
        """Get cast information for a drama. Primarily to access: Main Role cast."""
        return self._make_request(f"/id/{slug}/cast")

    def get_reviews(self, slug: str) -> Optional[Dict]:
        """Get reviews for a drama. Primarily to access: review text and number of people who found each review helpful."""
        return self._make_request(f"/id/{slug}/reviews")
        
    def get_user_dramalist(self, user_id: str) -> Optional[Dict]:
        """Get user's drama list. Primarily to access: rating of watched dramas."""
        return self._make_request(f"/dramalist/{user_id}")

# Initialize the API client
api = KuryanaAPI()

In [5]:
def extract_slugs_from_html_files(folder_path='html_popular'):
    """
    Extract drama slugs from all popular*.html files in the specified folder. 
    Files contain the 1000 most popular dramas based on MyDramaList.com's rankings on July 17, 2025. 
    
    Args:
        folder_path (str): Path to the folder containing HTML files
        
    Returns:
        list: All drama slugs found across all files
    """
    all_slugs = set()
    
    # Loop through all 50 files
    for i in range(1, 51):
        filename = f'popular{i}.html'
        filepath = os.path.join(folder_path, filename)
        
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                
                # Find all h6 elements with class "text-primary title"
                title_elements = soup.find_all('h6', class_='text-primary title')
                
                for title_element in title_elements:
                    # Find the anchor tag within this title element
                    link = title_element.find('a', href=True)
                    if link:
                        href = link['href']
                        # Remove the leading '/' to get just the slug
                        slug = href[1:]  # Remove the first character '/'
                        all_slugs.add(slug)
                        
        except FileNotFoundError:
            print(f"Warning: {filename} not found")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    
    return all_slugs

# Execute the function to get all slugs
popular_slugs = extract_slugs_from_html_files()

# Display results
print(f"Total slugs found: {len(popular_slugs)}")
# print(slugs)


Total slugs found: 1000


In [6]:
# # Get slugs from popular dramas via popular user created lists (will include some less popular dramas in the process)

# list_ids = ["3WW5GjX3","LlPQ0xJL","4vGPaBZ1","19BvZ0eL","1gnRZX74","Lp6p5Xn4","1NkpVODL","3m6VrKD4","71gkbY71","Ln89Aow4","L60RJaq4",
#             "389xkZO4","MLOPW2Z3","73Dxr8J3"]
# slugs = set()
# for id in list_ids:
#     dct = api.get_list(id)
#     lst = dct.get('data').get('list')
#     for drama in lst:
#         raw_slug = drama.get('slug')
#         slug = raw_slug[1:]
#         slugs.add(slug)

In [7]:
# ["D45GkMML","Y4BjGM04","LQJ77Nw3","MLOPrqD3"]

In [8]:
# print(len(slugs))
# print(slugs)

In [9]:
def get_watched_slugs(user_id='Oamen'):
    """
    Get slugs of watched dramas from a user's watchlist on the MyDramaList website.
    
    This function retrieves a user's completed drama list from an API.
    
    Args:
        user_id (str, optional): The user ID to check watched dramas for. 
                                Defaults to 'Oamen'.
    
    Returns:
        set: A set of drama slugs containing dramas that the user has completed.
    """
    watched_slugs = set()

    user_dramalist = api.get_user_dramalist(user_id)
    completed_dramas = user_dramalist.get('data').get('list').get('Completed').get('items')
    
    for drama in completed_dramas:
        slug = drama.get('id')
        watched_slugs.add(slug)
    
    return watched_slugs

In [10]:
watched_slugs =  get_watched_slugs()
unwatched_slugs = popular_slugs - watched_slugs
print(f"Total slugs remaining: {len(unwatched_slugs)}")

Total slugs remaining: 943


In [22]:
slug = "10904-descendants-of-the-sun"
dct = api.get_drama_info(slug)
data = dct.get('data')

genres = data.get('others').get('genres')

raw_synopsis = data.get('synopsis')
splits = raw_synopsis.split('\n(')
synopsis = splits[0]

raw_tags = data.get('others').get('tags')
new = raw_tags[-1:][0][:-12]
tags = raw_tags[:-1]
tags.append(new)
print(tags)

print(genres)
print(synopsis)

print(json.dumps(dct,indent=2))

['Military', 'Bromance', 'Multiple Couples', 'Hardworking Male Lead', 'Medical', 'Gun Violence', 'Hardworking Female Lead', 'Father-Daughter Relationship', 'Friendship', 'Filmed Abroad']
['Action', 'Comedy', 'Romance', 'Melodrama']
A love story that develops between a surgeon and a special forces officer.

Kang Mo Yeon is a pretty and assertive woman who works as a cardiothoracic surgeon at Haesung Hospital. She isn't afraid to admit her mistakes and believes that capability overrides whatever connections you have. However, she is soon faced with the reality that she cannot advance with just capability. Her life is forever changed when she encounters Yoo Si Jin, the Captain and team leader of the Alpha Team who cares more about protecting anybody in need of help as well as his country, even if it goes against the order of his superiors. 

It tells the story of how they both bond together in a time of war and overcome the odds against them.

{
  "slug_query": "10904-descendants-of-the-s

In [12]:
user_dramalist = api.get_user_dramalist('Oamen')
completed_dramas = user_dramalist.get('data').get('list').get('Completed').get('items')
test = user_dramalist.get('data').get('list')
user_dramalist

{'slug_query': 'dramalist/Oamen',
 'data': {'link': 'https://mydramalist.com/dramalist/Oamen',
  'list': {'Currently Watching': {'items': [{'name': 'Our Unwritten Seoul',
      'id': '762921-unknown-seoul',
      'score': '0.0',
      'episode_seen': '9',
      'episode_total': '12'}],
    'stats': {'Dramas': '1',
     'TV Shows': '0',
     'Episodes': '9',
     'Movies': '0',
     'Days': '0.5'}},
   'Completed': {'items': [{'name': '1987: When the Day Comes',
      'id': '21985-1987',
      'score': '5.0',
      'episode_seen': '1',
      'episode_total': '1'},
     {'name': 'Alchemy of Souls',
      'id': '52939-can-this-person-be-translated',
      'score': '9.5',
      'episode_seen': '20',
      'episode_total': '20'},
     {'name': 'Alchemy of Souls Season 2: Light and Shadow',
      'id': '733261-alchemy-of-souls-part-2',
      'score': '9.5',
      'episode_seen': '10',
      'episode_total': '10'},
     {'name': 'Alice, the Final Weapon',
      'id': '711493-she-s-the-last-we

In [13]:
api.get_reviews("10904-descendants-of-the-sun")

{'slug_query': '10904-descendants-of-the-sun/reviews?page=1',
 'data': {'link': 'https://mydramalist.com/10904-descendants-of-the-sun/reviews?page=1',
  'title': 'Descendants of the Sun',
  'poster': 'https://i.mydramalist.com/GwODpt.jpg',
  'reviews': [{'reviewer': {'name': 'manicmuse',
     'user_link': 'https://mydramalist.com/profile/manicmuse',
     'user_image': 'https://i.mydramalist.com/GwODpt.jpg',
     'info': '564 people found this review helpful'},
    'review': ['This review may contain spoilers',
     "When I think of Descendants of the Sun, the first thing that comes to mind is cheese! This show is extra cheesy, and honestly I love cheese! Unfortunately, I am also kinda lactose intolerant, so after eating a certain amount of cheese I am left with regrets, and a stomach ache. That's exactly how I felt after watching the last episode. *Cue the rage of a thousand fans* \n\n\n\r\nI think the biggest problem I had with this show is that it tried to mix in three genres without

In [28]:
api.get_cast("52939-can-this-person-be-translated")

{'slug_query': '52939-can-this-person-be-translated/cast',
 'data': {'link': 'https://mydramalist.com/52939-can-this-person-be-translated/cast',
  'title': 'Alchemy of Souls',
  'poster': 'https://i.mydramalist.com/dJEzg_5m.jpg',
  'casts': {'Composer': [{'name': 'Nam Hye Seung',
     'profile_image': 'https://i.mydramalist.com/dJEzg_5m.jpg',
     'slug': '/people/45603-nam-hye-seung',
     'link': 'https://mydramalist.com/people/45603-nam-hye-seung'}],
   'Director': [{'name': 'Park Joon Hwa',
     'profile_image': 'https://i.mydramalist.com/X4opwm.jpg',
     'slug': '/people/16362-park-joon-hwa',
     'link': 'https://mydramalist.com/people/16362-park-joon-hwa'}],
   'Executive Producer': [{'name': 'Jang Jung Do',
     'profile_image': 'https://i.mydramalist.com/d8o4K_5m.jpg',
     'slug': '/people/21183-jang-jung-do',
     'link': 'https://mydramalist.com/people/21183-jang-jung-do'}],
   'Screenwriter': [{'name': 'Hong Mi Ran',
     'profile_image': 'https://i.mydramalist.com/7nxXnm

In [18]:
api.get_drama_info("57049-private-life")

{'slug_query': '57049-private-life',
 'data': {'link': 'https://mydramalist.com/57049-private-life',
  'title': 'Private Lives',
  'complete_title': 'Private Lives',
  'sub_title': '사생활 ‧ Drama ‧ 2020',
  'year': '2020',
  'rating': 7.4,
  'poster': 'https://i.mydramalist.com/6K5dp_4c.jpg?v=1',
  'synopsis': 'Swindlers come across a secret of the nation and try to reveal the secret. They must go up against a large company by using all of their skills.\nLee Jung Hwan is a team leader for the major corporation. He looks like an ordinary company employee, but he is a mysterious figure. Cha Joo Eun is a swindler, who has the looks of a sweet and innocent woman. She supports herself with her crimes. Jung Bok Ki is a professional swindler. She often targets other swindlers. She is elegant and also charismatic. Kim Jae Wook is Jung Bok Ki’s partner in crime.\n\n(Source: AsianWiki)',
  'casts': [{'name': 'Seo Hyun',
    'profile_image': 'https://i.mydramalist.com/PxB3DZ_5s.jpg',
    'slug': '/